In [10]:
from sqlalchemy import create_engine, Sequence, Column, Integer, String, Float, DateTime
from sqlalchemy.orm import sessionmaker, declarative_base
import polars as pl

In [11]:
Base = declarative_base()


In [12]:
# ORM model
class Order(Base):
    __tablename__ = 'orders'

    id = Column(Integer, Sequence('id'), primary_key=True)
    order_id = Column(String)
    customer_unique_id = Column(String)
    order_status = Column(String)
    order_purchase_timestamp = Column(DateTime)
    order_approved_at = Column(DateTime)
    order_delivered_carrier_date = Column(DateTime)
    order_delivered_customer_date = Column(DateTime)
    order_estimated_delivery_date = Column(DateTime)
    item_quantity = Column(Integer)
    price = Column(Float)
    freight_value = Column(Float)
    product_id = Column(String)
    product_category_name = Column(String)
    product_category_name_english = Column(String)
    customer_city = Column(String)
    seller_id = Column(String)
    seller_city = Column(String)
    payment_installments = Column(Integer)
    payment_value = Column(Float)
    review_score = Column(Integer)

In [17]:

def setup_database(db_path):
    engine = create_engine(f'duckdb:///{db_path}')
    Base.metadata.create_all(engine)
    return engine

def create_session(engine):
    Session = sessionmaker(bind=engine)
    return Session()

In [14]:
import os
WORKING_DIR = os.path.abspath('../data')
db_path = f'{WORKING_DIR}/orders.db'

# Load relevant CSV files
customers_df = pl.read_csv(f'{WORKING_DIR}/olist_customers_dataset.csv')
orders_df = pl.read_csv(f'{WORKING_DIR}/olist_orders_dataset.csv')
order_items_df = pl.read_csv(f'{WORKING_DIR}/olist_order_items_dataset.csv')
payments_df = pl.read_csv(f'{WORKING_DIR}/olist_order_payments_dataset.csv')
reviews_df = pl.read_csv(f'{WORKING_DIR}/olist_order_reviews_dataset.csv')
products_df = pl.read_csv(f'{WORKING_DIR}/olist_products_dataset.csv')
sellers_df = pl.read_csv(f'{WORKING_DIR}/olist_sellers_dataset.csv')
product_translation_df =pl.read_csv(f'{WORKING_DIR}/product_category_name_translation.csv')




In [15]:
# Join orders with customers
orders_with_customers = orders_df.join(customers_df, on='customer_id', how='left')

# Join order items with products and product name in english
order_items_with_products = order_items_df.join(products_df, on='product_id', how='left')

# Aggregate order items to get total price and quantity
order_items_agg = order_items_with_products.group_by('order_id').agg([
    pl.sum('price').alias('price'),
    pl.sum('freight_value').alias('freight_value'),
    pl.first('product_id'),
    pl.first('product_category_name'),
    pl.first('seller_id'),
    pl.count('order_item_id').alias('item_quantity')
])

# Aggregate payments to get total payments for each order
# (took out the payment type and sequential number because they are different for each payment)
payments_agg = payments_df.group_by('order_id').agg([
    pl.first('payment_installments'),
    pl.sum('payment_value').alias('payment_value')
])

# Get average review score by order_id
reviews_agg = reviews_df.group_by('order_id').agg([
    pl.mean('review_score').cast(pl.Int32).alias('review_score')
])

# Merge orders_with_customers and order_items_agg
orders_full = orders_with_customers.join(order_items_agg, on='order_id', how='left')


# Merge with payments
orders_full = orders_full.join(payments_agg, on='order_id', how='left')

# Merge with reviews and sellers
orders_full = orders_full.join(sellers_df, on='seller_id', how='left')

orders_full = orders_full.join(reviews_agg, on='order_id', how='left')

# Merge with product category name translation
orders_full = orders_full.join(product_translation_df, on='product_category_name', how='left')

# Final orders dataframe with necessary columns
final_orders_df = orders_full.select([
    'order_id',
    'customer_unique_id',
    'order_status',
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'customer_city',
    'price',
    'freight_value',
    'product_id',
    'product_category_name',
    'seller_id',
    'item_quantity',
    'payment_installments',
    'payment_value',
    'review_score',
    'seller_city',
    'product_category_name_english'
    ])

final_orders_df.describe()







statistic,order_id,customer_unique_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_city,price,freight_value,product_id,product_category_name,seller_id,item_quantity,payment_installments,payment_value,review_score,seller_city,product_category_name_english
str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,str,f64,f64,f64,f64,str,str
"""count""","""99441""","""99441""","""99441""","""99441""","""99281""","""97658""","""96476""","""99441""","""99441""",98666.0,98666.0,"""98666""","""97250""","""98666""",98666.0,99440.0,99440.0,98673.0,"""98666""","""97229"""
"""null_count""","""0""","""0""","""0""","""0""","""160""","""1783""","""2965""","""0""","""0""",775.0,775.0,"""775""","""2191""","""775""",775.0,1.0,1.0,768.0,"""775""","""2212"""
"""mean""",null,null,null,null,null,null,null,null,null,137.754076,22.823562,null,null,null,1.141731,2.915617,160.990267,4.086174,null,null
"""std""",null,null,null,null,null,null,null,null,null,210.645145,21.650909,null,null,null,0.538452,2.709861,221.951257,1.346642,null,null
"""min""","""00010242fe8c5a6d1ba2dd792cb162…","""0000366f3b9a7992bf8c76cfdf3221…","""approved""","""2016-09-04 21:15:19""","""2016-09-15 12:16:38""","""2016-10-08 10:34:01""","""2016-10-11 13:46:32""","""2016-09-30 00:00:00""","""abadia dos dourados""",0.85,0.0,"""00066f42aeeb9f3007548bb9d3f33c…","""agro_industria_e_comercio""","""0015a82c2db000af6aaaf3ae2ecb05…",1.0,0.0,0.0,1.0,"""4482255""","""agro_industry_and_commerce"""
"""25%""",null,null,null,null,null,null,null,null,null,45.9,13.85,null,null,null,1.0,1.0,62.01,4.0,null,null
"""50%""",null,null,null,null,null,null,null,null,null,86.9,17.17,null,null,null,1.0,2.0,105.29,5.0,null,null
"""75%""",null,null,null,null,null,null,null,null,null,149.9,24.04,null,null,null,1.0,4.0,176.97,5.0,null,null
"""max""","""fffe41c64501cc87c801fd61db3f62…","""ffffd2657e2aad2907e67c3e9daecb…","""unavailable""","""2018-10-17 17:30:18""","""2018-09-03 17:40:06""","""2018-09-11 19:48:28""","""2018-10-17 13:22:46""","""2018-11-12 00:00:00""","""zortea""",13440.0,1794.96,"""fffe9eeff12fcbd74a2f2b007dde0c…","""utilidades_domesticas""","""ffff564a4f9085cd26170f47323937…",21.0,24.0,13664.08,5.0,"""xaxim""","""watches_gifts"""


In [19]:

# Setup the database and create an engine. 
engine = setup_database(db_path)

# Create a session
session = create_session(engine)

try:
    # convert dataframe to a list of dictionaries -> converts each row to a dictonary
    records = final_orders_df.to_dicts()

    # Create instances of Record from records dict and inserts each into the database
    for record in records:
        order = Order(**record)
        session.add(order)

    # Commit the transaction
    session.commit()

except Exception as e:
    # Catch any type of exception
    print(f"An error occurred: {e}")
    session.rollback()  # Rollback the transaction to avoid partial commits

finally:
    # Close the session
    session.close()


In [20]:
# Query the database
session = create_session(engine)
query = session.query(Order).filter(Order.order_status == 'delivered').limit(5)

for order in query:
    print(order.order_id, order.customer_unique_id, order.order_status) # Print the order_id, customer_unique_id and order_status
    

e481f51cbdc54678b7cc49136f2d6af7 7c396fd4830fd04220f754e42b4e5bff delivered
53cdb2fc8bc7dce0b6741e2150273451 af07308b275d755c9edb36a90c618231 delivered
47770eb9100c2d0c44946d9cf07ec65d 3a653a41f6f9fc3d2a113cf8398680e8 delivered
949d5b44dbf5de918fe9c16f97b45f8a 7c142cf63193a1473d2e66489a9ae977 delivered
ad21c59c0840e6cb83a9ceb5573f8159 72632f0f9dd73dfee390c9b22eb56dd6 delivered
